In [57]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
import configparser
import csv
import pandas as pd
import numpy as np 
import tweepy
config = configparser.ConfigParser()
config.read('../config.ini')
consumer_key = config['DEFAULT']['TWITTER_CONSUMER_KEY']
consumer_secret_key = config['DEFAULT']['TWITTER_CONSUMER_SECRET_KEY']
access_token = config['DEFAULT']['TWITTER_ACCESS_TOKEN']
access_token_secret = config['DEFAULT']['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [10]:
data_file = os.path.join(ROOT_DIR, 'data/founta_data.csv')

In [19]:
data = pd.read_csv(data_file, header='infer', index_col=0, squeeze=True).to_dict()

In [20]:
print(len(data))

99799


In [103]:
for class_name in ['abusive', 'hateful', 'spam', 'normal']:
    print("{} {:0.2f}%".format(class_name, (list(data.values()).count(class_name) / len(data)) * 100))

abusive 27.09%
hateful 4.96%
normal 53.90%


In [48]:
status_ids = list(data.keys())
status_ids[0]

849667487180259329

In [62]:
def fetch_founta_data():
    founta_data = np.load(os.path.join(ROOT_DIR, 'data/founta_data.npy'))
    founta_data = founta_data[()]
    return founta_data

In [79]:
# start_ptr = 0
# end_ptr = start_ptr + 100 
# while(start_ptr <= len(status_ids)):
#     print("Start ptr is at {}".format(start_ptr))
#     tweets = api.statuses_lookup(status_ids[start_ptr:end_ptr])
    
#     # load and save results each time 
#     if os.path.exists(os.path.join(ROOT_DIR, 'data/founta_data.npy')):
#         fetch_founta_data()
#     else: 
#         founta_data = {}
#     for i, tweet in enumerate(tweets):
#         tweet = tweet._json
#         founta_data[tweet['id_str']] = tweet
#     np.save(os.path.join(ROOT_DIR, 'data/founta_data.npy'), founta_data)
    
#     start_ptr += 100 
#     end_ptr += 100

Start ptr is at 0
Start ptr is at 100
Start ptr is at 200
Start ptr is at 300
Start ptr is at 400
Start ptr is at 500
Start ptr is at 600
Start ptr is at 700
Start ptr is at 800
Start ptr is at 900
Start ptr is at 1000
Start ptr is at 1100
Start ptr is at 1200
Start ptr is at 1300
Start ptr is at 1400
Start ptr is at 1500
Start ptr is at 1600
Start ptr is at 1700
Start ptr is at 1800
Start ptr is at 1900
Start ptr is at 2000
Start ptr is at 2100
Start ptr is at 2200
Start ptr is at 2300
Start ptr is at 2400
Start ptr is at 2500
Start ptr is at 2600
Start ptr is at 2700
Start ptr is at 2800
Start ptr is at 2900
Start ptr is at 3000
Start ptr is at 3100
Start ptr is at 3200
Start ptr is at 3300
Start ptr is at 3400
Start ptr is at 3500
Start ptr is at 3600
Start ptr is at 3700
Start ptr is at 3800
Start ptr is at 3900
Start ptr is at 4000
Start ptr is at 4100
Start ptr is at 4200
Start ptr is at 4300
Start ptr is at 4400
Start ptr is at 4500
Start ptr is at 4600
Start ptr is at 4700
Star

Start ptr is at 37800
Start ptr is at 37900
Start ptr is at 38000
Start ptr is at 38100
Start ptr is at 38200
Start ptr is at 38300
Start ptr is at 38400
Start ptr is at 38500
Start ptr is at 38600
Start ptr is at 38700
Start ptr is at 38800
Start ptr is at 38900
Start ptr is at 39000
Start ptr is at 39100
Start ptr is at 39200
Start ptr is at 39300
Start ptr is at 39400
Start ptr is at 39500
Start ptr is at 39600
Start ptr is at 39700
Start ptr is at 39800
Start ptr is at 39900
Start ptr is at 40000
Start ptr is at 40100
Start ptr is at 40200
Start ptr is at 40300
Start ptr is at 40400
Start ptr is at 40500
Start ptr is at 40600
Start ptr is at 40700
Start ptr is at 40800
Start ptr is at 40900
Start ptr is at 41000
Start ptr is at 41100
Start ptr is at 41200
Start ptr is at 41300
Start ptr is at 41400
Start ptr is at 41500
Start ptr is at 41600
Start ptr is at 41700
Start ptr is at 41800
Start ptr is at 41900
Start ptr is at 42000
Start ptr is at 42100
Start ptr is at 42200
Start ptr 

Start ptr is at 75100
Start ptr is at 75200
Start ptr is at 75300
Start ptr is at 75400
Start ptr is at 75500
Start ptr is at 75600
Start ptr is at 75700
Start ptr is at 75800
Start ptr is at 75900
Start ptr is at 76000
Start ptr is at 76100
Start ptr is at 76200
Start ptr is at 76300
Start ptr is at 76400
Start ptr is at 76500
Start ptr is at 76600
Start ptr is at 76700
Start ptr is at 76800
Start ptr is at 76900
Start ptr is at 77000
Start ptr is at 77100
Start ptr is at 77200
Start ptr is at 77300
Start ptr is at 77400
Start ptr is at 77500
Start ptr is at 77600
Start ptr is at 77700
Start ptr is at 77800
Start ptr is at 77900
Start ptr is at 78000
Start ptr is at 78100
Start ptr is at 78200
Start ptr is at 78300
Start ptr is at 78400
Start ptr is at 78500
Start ptr is at 78600
Start ptr is at 78700
Start ptr is at 78800
Start ptr is at 78900
Start ptr is at 79000
Start ptr is at 79100
Start ptr is at 79200
Start ptr is at 79300
Start ptr is at 79400
Start ptr is at 79500
Start ptr 

In [80]:
founta_data = fetch_founta_data()

In [82]:
len(founta_data.keys())

64149

In [83]:
64149/99799

0.6427819918035251

In [89]:
list(founta_data.keys())[0]

'848337741813358592'

In [90]:
founta_data['848337741813358592']

{'created_at': 'Sun Apr 02 00:54:13 +0000 2017',
 'id': 848337741813358592,
 'id_str': '848337741813358592',
 'text': 'fucks sake go away stupid anon — ^  https://t.co/8TQGyiKCVE',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/8TQGyiKCVE',
    'expanded_url': 'https://curiouscat.me/christiangrcy/post/128558256?t=1491094453',
    'display_url': 'curiouscat.me/christiangrcy/…',
    'indices': [36, 59]}]},
 'source': '<a href="https://curiouscat.me" rel="nofollow">Curious Cat</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 2892845505,
  'id_str': '2892845505',
  'name': 'jeremy',
  'screen_name': 'fIoralhearts',
  'location': '',
  'description': 'shine until tomorrow, let it be 🌻',
  'url': 'https://t.co/Ve1HqASFu2',
  'entities': {'url': {'urls': [{'url': 'https://t.co/Ve1

In [134]:
labels = [data[int(key)] for key in founta_data.keys() if int(key) in data and data[int(key)] != 'spam']
len(labels)

55306

In [137]:
for class_name in ['abusive', 'hateful', 'normal']:
    print("{} {:0.2f}%".format(class_name, (labels.count(class_name) / len(labels)) * 100))
len(labels)

abusive 23.22%
hateful 4.86%
normal 71.92%


55306

In [120]:
my_labels_file = os.path.join(ROOT_DIR, 'data/labels.csv')
my_labels = pd.read_csv(data_file, header='infer', index_col=0, squeeze=True).to_dict()
list(my_labels.keys())[0]

849667487180259329

In [128]:
import json
my_data_file = os.path.join(ROOT_DIR, 'data/80k_tweets.json')
all_my_labels = []
with open(my_data_file, 'r') as f:
    for line in f.readlines():
        obj = json.loads(line)
        status_id = int(obj['id_str'])
        if status_id not in my_labels:
            print("err")
        else:
            if my_labels[status_id] == 'spam':
                continue
            all_my_labels.append(my_labels[status_id])

err
err
err


In [129]:
for class_name in ['abusive', 'hateful', 'spam', 'normal']:
    print("{} {:0.2f}%".format(class_name, (all_my_labels.count(class_name) / len(all_my_labels)) * 100))

abusive 11.97%
hateful 4.75%
spam 0.00%
normal 83.28%


In [130]:
print(len(all_my_labels))

49194


In [180]:
## Determine that this new data is better 
from collections import Counter

retweet_count_hateful = []
retweet_count_normal = []
statuses = []
user_ids = []
for key, value in founta_data.items():
    label = data[value['id']]
    if label == 'hateful':
        retweet_count_hateful.append(1 if value['in_reply_to_status_id'] else 0)
    else:
        retweet_count_normal.append(1 if value['in_reply_to_status_id'] else 0)


In [186]:
np.mean(retweet_count_hateful)

0.25344250093040566

In [187]:
np.mean(retweet_count_normal)

0.1887507728352478

In [189]:
(25 - 19) / 19

0.3157894736842105

In [168]:
(13 - 3)/ 13

0.7692307692307693

In [163]:
245 * 1.142

279.78999999999996